##### Plan 

- show gradients of gather with feature correlation
- write my own
- test

In [18]:
import os; os.environ['CUDA_VISIBLE_DEVICES'] = ""
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from timeit import default_timer as timer

In [19]:
sess = tf.Session()

In [20]:
size=(10,3)
np.random.seed(42)
lf = tf.convert_to_tensor(np.random.rand(*size).astype(np.float32))
rf = tf.convert_to_tensor(np.random.rand(*size).astype(np.float32))
hi = tf.convert_to_tensor(np.random.randint(0, size[0], size=size[0]).astype(np.int32))
sess.run([lf, rf, hi])

[array([[0.37454012, 0.9507143 , 0.7319939 ],
        [0.5986585 , 0.15601864, 0.15599452],
        [0.05808361, 0.8661761 , 0.601115  ],
        [0.7080726 , 0.02058449, 0.96990985],
        [0.83244264, 0.21233912, 0.18182497],
        [0.1834045 , 0.30424225, 0.52475643],
        [0.43194503, 0.29122913, 0.6118529 ],
        [0.13949387, 0.29214466, 0.36636186],
        [0.45606998, 0.785176  , 0.19967379],
        [0.5142344 , 0.59241456, 0.04645041]], dtype=float32),
 array([[0.60754484, 0.17052412, 0.06505159],
        [0.94888556, 0.965632  , 0.80839735],
        [0.30461377, 0.09767211, 0.684233  ],
        [0.4401525 , 0.12203824, 0.4951769 ],
        [0.03438852, 0.9093204 , 0.25877997],
        [0.66252226, 0.31171107, 0.52006805],
        [0.54671025, 0.18485446, 0.96958464],
        [0.77513283, 0.93949896, 0.89482737],
        [0.5979    , 0.9218742 , 0.08849251],
        [0.19598286, 0.04522729, 0.32533032]], dtype=float32),
 array([1, 4, 7, 9, 8, 8, 0, 8, 6, 8], dtype=i

In [21]:
def composed(sparams, params, indices):
    moved = tf.gather(params, indices)
    corr = tf.reduce_sum(tf.multiply(sparams, moved), axis=1)
    loss = tf.square(corr)
    return loss

lib = Path.home() / 'tensorflow/bazel-bin/tensorflow/core/user_ops/gather_corr.so'
assert(lib.exists())
gather_module = tf.load_op_library(str(lib))
print(gather_module.OP_LIST)
gather_corr = gather_module.gather_corr
from gather_corr_grad import *

def composed_new(sparams, params, indices):
    moved = gather_corr(params, indices)
    corr = tf.reduce_sum(tf.multiply(sparams, moved), axis=1)
    loss = tf.square(corr)
    return loss

op {
  name: "GatherCorr"
  input_arg {
    name: "params"
    type_attr: "Tparams"
  }
  input_arg {
    name: "indices"
    type_attr: "Tindices"
  }
  output_arg {
    name: "output"
    type_attr: "Tparams"
  }
  attr {
    name: "validate_indices"
    type: "bool"
    default_value {
      b: true
    }
  }
  attr {
    name: "Tparams"
    type: "type"
  }
  attr {
    name: "Tindices"
    type: "type"
    allowed_values {
      list {
        type: DT_INT32
        type: DT_INT64
      }
    }
  }
}
op {
  name: "GatherCorrV2"
  input_arg {
    name: "params"
    type_attr: "Tparams"
  }
  input_arg {
    name: "indices"
    type_attr: "Tindices"
  }
  input_arg {
    name: "axis"
    type_attr: "Taxis"
  }
  output_arg {
    name: "output"
    type_attr: "Tparams"
  }
  attr {
    name: "Tparams"
    type: "type"
  }
  attr {
    name: "Tindices"
    type: "type"
    allowed_values {
      list {
        type: DT_INT32
        type: DT_INT64
      }
    }
  }
  attr {
    name: 

In [22]:
m = np.multiply(np.array([0.37454012, 0.9507143 , 0.7319939 ]), np.array([0.94888556, 0.965632  , 0.80839735]))
m, np.sum(m)

(array([0.35539571, 0.91804015, 0.59174193]), 1.8651777914224321)

In [23]:
def run_example(lf, rf, hi, func, its=100):
    seed = 0
    np.random.seed(seed)
    tf.set_random_seed(seed)
    init = tf.initializers.random_normal(seed=0)
    dlf1 = tf.layers.dense(lf, 100, kernel_initializer=init)
    drf1 = tf.layers.dense(rf, 100, kernel_initializer=init)    
    dlf1 = tf.layers.dense(dlf1, 100, kernel_initializer=init)
    drf1 = tf.layers.dense(drf1, 100, kernel_initializer=init)
    dhi1 = hi
    loss1 = func(dlf1, drf1, dhi1)
    opt1 = tf.train.AdamOptimizer(0.0001).minimize(loss1)
    sess.run(tf.global_variables_initializer())
    losses = []
    for i in range(its):
        d = sess.run({'opt1':opt1, 'loss1':loss1})
        losses.append(d['loss1'])
    return losses

In [24]:
res1 = run_example(lf, rf, hi, composed)
res2 = run_example(lf, rf, hi, composed)
res3 = run_example(lf, rf, hi, composed_new)


prec = [[np.abs(x) for x in [a1-a2,a1-a3]] for i, (a1, a2, a3) in enumerate(zip(res1, res2, res3))]
prec_ok, prec_asked = np.array(prec).sum(axis=2).sum(axis=0)
print(prec_ok, prec_asked)

534.0 945.5
